In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv( "C:/Users/Abhi/Desktop/Python_Data_Analytics_Projects/content_engagement_churn_analytics/data/raw/content_engagement_raw.csv",parse_dates=["session_date", "last_active_date"])

In [7]:
df.head()

,user_id,session_date,content_id,content_type,watch_minutes,session_count,subscription_plan,ad_views,revenue,last_active_date,churn_flag
0,U00001,2023-04-13,C2827,Movie,70,1,Basic,0,0.25,2024-03-22,1
1,U00001,2023-04-14,C9569,Kids,82,2,Basic,0,0.25,2024-03-22,1
2,U00001,2023-04-14,C3196,Series,14,2,Basic,0,0.25,2024-03-22,1
3,U00001,2023-04-15,C7303,Series,17,3,Basic,0,0.25,2024-03-22,1
4,U00001,2023-04-15,C9714,Kids,102,3,Basic,0,0.25,2024-03-22,1


In [9]:
df = df.sort_values(["user_id", "session_date"])
df.head()

,user_id,session_date,content_id,content_type,watch_minutes,session_count,subscription_plan,ad_views,revenue,last_active_date,churn_flag
0,U00001,2023-04-13,C2827,Movie,70,1,Basic,0,0.25,2024-03-22,1
1,U00001,2023-04-14,C9569,Kids,82,2,Basic,0,0.25,2024-03-22,1
2,U00001,2023-04-14,C3196,Series,14,2,Basic,0,0.25,2024-03-22,1
3,U00001,2023-04-15,C7303,Series,17,3,Basic,0,0.25,2024-03-22,1
4,U00001,2023-04-15,C9714,Kids,102,3,Basic,0,0.25,2024-03-22,1


In [11]:
daily_df = (
    df.groupby(["user_id", "session_date", "subscription_plan", "churn_flag"])
      .agg(
          total_watch_minutes=("watch_minutes", "sum"),
          total_sessions=("session_count", "sum"),
          total_ads=("ad_views", "sum"),
          daily_revenue=("revenue", "sum")
      )
      .reset_index()
)


In [15]:
daily_df.head()

,user_id,session_date,subscription_plan,churn_flag,total_watch_minutes,total_sessions,total_ads,daily_revenue
0,U00001,2023-04-13,Basic,1,70,1,0,0.25
1,U00001,2023-04-14,Basic,1,96,4,0,0.50
2,U00001,2023-04-15,Basic,1,138,9,0,0.75
3,U00001,2023-04-16,Basic,1,204,9,0,0.75
4,U00001,2023-04-17,Basic,1,36,1,0,0.25


In [17]:
daily_df["watch_7d_avg"] = (
    daily_df.groupby("user_id")["total_watch_minutes"]
            .transform(lambda x: x.rolling(7, min_periods=1).mean())
)
daily_df.head()

,user_id,session_date,subscription_plan,churn_flag,total_watch_minutes,total_sessions,total_ads,daily_revenue,watch_7d_avg
0,U00001,2023-04-13,Basic,1,70,1,0,0.25,70.000000
1,U00001,2023-04-14,Basic,1,96,4,0,0.50,83.000000
2,U00001,2023-04-15,Basic,1,138,9,0,0.75,101.333333
3,U00001,2023-04-16,Basic,1,204,9,0,0.75,127.000000
4,U00001,2023-04-17,Basic,1,36,1,0,0.25,108.800000


In [23]:
daily_df["watch_30d_avg"] = (
    daily_df.groupby("user_id")["total_watch_minutes"]
            .transform(lambda x: x.rolling(30, min_periods=1).mean())
)
daily_df.head(15)

,user_id,session_date,subscription_plan,churn_flag,total_watch_minutes,total_sessions,total_ads,daily_revenue,watch_7d_avg,watch_30d_avg
0,U00001,2023-04-13,Basic,1,70,1,0,0.25,70.000000,70.000000
1,U00001,2023-04-14,Basic,1,96,4,0,0.50,83.000000,83.000000
2,U00001,2023-04-15,Basic,1,138,9,0,0.75,101.333333,101.333333
3,U00001,2023-04-16,Basic,1,204,9,0,0.75,127.000000,127.000000
4,U00001,2023-04-17,Basic,1,36,1,0,0.25,108.800000,108.800000
5,U00001,2023-04-18,Basic,1,103,1,0,0.25,107.833333,107.833333
6,U00001,2023-04-19,Basic,1,53,9,0,0.75,100.000000,100.000000
7,U00001,2023-04-20,Basic,1,193,4,0,0.50,117.571429,111.625000
8,U00001,2023-04-21,Basic,1,86,1,0,0.25,116.142857,108.777778
9,U00001,2023-04-22,Basic,1,245,9,0,0.75,131.428571,122.400000


In [25]:
daily_df["engagement_volatility"] = (
    daily_df.groupby("user_id")["total_watch_minutes"]
            .transform(lambda x: x.rolling(14, min_periods=3).std())
)
daily_df.head(15)

,user_id,session_date,subscription_plan,churn_flag,total_watch_minutes,total_sessions,total_ads,daily_revenue,watch_7d_avg,watch_30d_avg,engagement_volatility
0,U00001,2023-04-13,Basic,1,70,1,0,0.25,70.000000,70.000000,NaN
1,U00001,2023-04-14,Basic,1,96,4,0,0.50,83.000000,83.000000,NaN
2,U00001,2023-04-15,Basic,1,138,9,0,0.75,101.333333,101.333333,34.312291
3,U00001,2023-04-16,Basic,1,204,9,0,0.75,127.000000,127.000000,58.480766
4,U00001,2023-04-17,Basic,1,36,1,0,0.25,108.800000,108.800000,64.970763
5,U00001,2023-04-18,Basic,1,103,1,0,0.25,107.833333,107.833333,58.159837
6,U00001,2023-04-19,Basic,1,53,9,0,0.75,100.000000,100.000000,56.994152
7,U00001,2023-04-20,Basic,1,193,4,0,0.50,117.571429,111.625000,62.172427
8,U00001,2023-04-21,Basic,1,86,1,0,0.25,116.142857,108.777778,58.780902
9,U00001,2023-04-22,Basic,1,245,9,0,0.75,131.428571,122.400000,70.192117


In [27]:
daily_df["sessions_per_day"] = daily_df["total_sessions"]
daily_df.head()

,user_id,session_date,subscription_plan,churn_flag,total_watch_minutes,total_sessions,total_ads,daily_revenue,watch_7d_avg,watch_30d_avg,engagement_volatility,sessions_per_day
0,U00001,2023-04-13,Basic,1,70,1,0,0.25,70.000000,70.000000,NaN,1
1,U00001,2023-04-14,Basic,1,96,4,0,0.50,83.000000,83.000000,NaN,4
2,U00001,2023-04-15,Basic,1,138,9,0,0.75,101.333333,101.333333,34.312291,9
3,U00001,2023-04-16,Basic,1,204,9,0,0.75,127.000000,127.000000,58.480766,9
4,U00001,2023-04-17,Basic,1,36,1,0,0.25,108.800000,108.800000,64.970763,1


In [29]:
daily_df["revenue_30d"] = (
    daily_df.groupby("user_id")["daily_revenue"]
            .transform(lambda x: x.rolling(30, min_periods=1).sum())
)
daily_df.head()

,user_id,session_date,subscription_plan,churn_flag,total_watch_minutes,total_sessions,total_ads,daily_revenue,watch_7d_avg,watch_30d_avg,engagement_volatility,sessions_per_day,revenue_30d
0,U00001,2023-04-13,Basic,1,70,1,0,0.25,70.000000,70.000000,NaN,1,0.25
1,U00001,2023-04-14,Basic,1,96,4,0,0.50,83.000000,83.000000,NaN,4,0.75
2,U00001,2023-04-15,Basic,1,138,9,0,0.75,101.333333,101.333333,34.312291,9,1.50
3,U00001,2023-04-16,Basic,1,204,9,0,0.75,127.000000,127.000000,58.480766,9,2.25
4,U00001,2023-04-17,Basic,1,36,1,0,0.25,108.800000,108.800000,64.970763,1,2.50


In [31]:
daily_df["engagement_change"] = (
    daily_df.groupby("user_id")["watch_7d_avg"]
            .diff()
)
daily_df.head()

,user_id,session_date,subscription_plan,churn_flag,total_watch_minutes,total_sessions,total_ads,daily_revenue,watch_7d_avg,watch_30d_avg,engagement_volatility,sessions_per_day,revenue_30d,engagement_change
0,U00001,2023-04-13,Basic,1,70,1,0,0.25,70.000000,70.000000,NaN,1,0.25,NaN
1,U00001,2023-04-14,Basic,1,96,4,0,0.50,83.000000,83.000000,NaN,4,0.75,13.000000
2,U00001,2023-04-15,Basic,1,138,9,0,0.75,101.333333,101.333333,34.312291,9,1.50,18.333333
3,U00001,2023-04-16,Basic,1,204,9,0,0.75,127.000000,127.000000,58.480766,9,2.25,25.666667
4,U00001,2023-04-17,Basic,1,36,1,0,0.25,108.800000,108.800000,64.970763,1,2.50,-18.200000


In [33]:
user_features = (
    daily_df.groupby("user_id")
    .agg(
        subscription_plan=("subscription_plan", "last"),
        churn_flag=("churn_flag", "max"),
        avg_watch_minutes=("total_watch_minutes", "mean"),
        avg_sessions=("sessions_per_day", "mean"),
        engagement_volatility=("engagement_volatility", "mean"),
        avg_watch_7d=("watch_7d_avg", "mean"),
        avg_watch_30d=("watch_30d_avg", "mean"),
        revenue_30d=("revenue_30d", "mean"),
        engagement_trend=("engagement_change", "mean")
    )
    .reset_index()
)


In [37]:
user_features.head()

,user_id,subscription_plan,churn_flag,avg_watch_minutes,avg_sessions,engagement_volatility,avg_watch_7d,avg_watch_30d,revenue_30d,engagement_trend
0,U00001,Basic,1,122.327536,4.547826,71.764339,122.122885,122.184636,14.140580,0.143272
1,U00002,Basic,1,121.375000,4.601562,63.786368,121.955407,121.521901,14.363281,-0.337561
2,U00003,Basic,1,124.544828,4.583908,67.809904,123.947411,122.452469,14.261494,0.251152
3,U00004,Free,1,128.353247,4.927273,69.014366,128.166357,127.670915,5.371558,-0.169271
4,U00005,Basic,1,121.992167,4.738903,63.305509,121.290066,120.139129,14.542428,0.096485


In [39]:
user_features["engagement_score"] = (
    0.4 * user_features["avg_watch_minutes"] +
    0.3 * user_features["avg_sessions"] -
    0.3 * user_features["engagement_volatility"]
)


In [41]:
user_features.head()


,user_id,subscription_plan,churn_flag,avg_watch_minutes,avg_sessions,engagement_volatility,avg_watch_7d,avg_watch_30d,revenue_30d,engagement_trend,engagement_score
0,U00001,Basic,1,122.327536,4.547826,71.764339,122.122885,122.184636,14.140580,0.143272,28.766061
1,U00002,Basic,1,121.375000,4.601562,63.786368,121.955407,121.521901,14.363281,-0.337561,30.794558
2,U00003,Basic,1,124.544828,4.583908,67.809904,123.947411,122.452469,14.261494,0.251152,30.850132
3,U00004,Free,1,128.353247,4.927273,69.014366,128.166357,127.670915,5.371558,-0.169271,32.115171
4,U00005,Basic,1,121.992167,4.738903,63.305509,121.290066,120.139129,14.542428,0.096485,31.226885


In [43]:
user_features.describe()


,churn_flag,avg_watch_minutes,avg_sessions,engagement_volatility,avg_watch_7d,avg_watch_30d,revenue_30d,engagement_trend,engagement_score
count,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,0.996667,123.898592,4.665740,67.984892,123.912094,123.940216,11.766565,-0.005455,30.563691
std,0.057663,4.072325,0.196041,2.695666,4.111945,4.411288,6.745921,0.327023,1.574874
min,0.000000,107.032609,3.965854,56.453211,105.294565,107.624593,4.182424,-1.933426,24.591266
25%,1.000000,121.198526,4.545732,66.361670,121.295349,121.156163,5.193179,-0.149223,29.501879
50%,1.000000,123.763625,4.664760,67.944348,123.837041,123.691430,13.530654,0.013105,30.539551
75%,1.000000,126.572198,4.785018,69.613987,126.605087,126.730116,14.654733,0.165364,31.543867
max,1.000000,139.810000,5.431579,78.994337,142.719905,145.419393,24.408861,1.619481,36.653170


In [45]:
user_features["churn_flag"].value_counts()


churn_flag
1    1196
0       4
Name: count, dtype: int64

In [47]:
user_features.to_csv(
     "C:/Users/Abhi/Desktop/Python_Data_Analytics_Projects/content_engagement_churn_analytics/data/processed/engagement_features.csv",
    index=False
)
